### HinetPyを用いて波形データを取得する

In [2]:
import os
import configparser
from HinetPy import Client, win32


config = configparser.ConfigParser()
config.read('config.ini')

# クライアントの初期化
client = Client(
    user=config['hinet']['username'],
    password=config['hinet']['password']
)

code = "0203"
minites = 10
#### 観測点名で指定
# client.select_station("0101",['N.RBSH','N.KKKH'])
##### 防災科研以外は組織コードを指定するだけ
# client.select_station("0201")

#### 緯度経度で指定
# client.select_stations('0101', minlatitude=40, maxlatitude=50, minlongitude=140, maxlongitude=150)

#### 任意の地点と半径で指定
# client.select_stations('0101', latitude=30, longitude=139, minradius=0, maxradius=2)


In [3]:
print(f"./waveform/{code}_202503010000_{minites}.cnt")


./waveform/0203_202503010000_10.cnt


In [4]:
!echo "./waveform/{code}_202503010000_{minites}.cnt"


./waveform/0203_202503010000_10.cnt


In [5]:
if not os.path.exists(f"./waveform/{code}_202503010000_{minites}.cnt"):
    data, ctable = client.get_continuous_waveform(code,"202503010000",minites)
    # データとチャンネル表をwaveformディレクトリに移動
    os.makedirs("waveform", exist_ok=True)


In [6]:
win_file = "025030100.00"
!w32tow1 "/workspaces/base-devcontainer/waveform/0203_202503010000_10.cnt" "./waveform/25030100.00"

In [13]:
# import obspy
# st = obspy.read(f"./waveform/{win_file[1:]}", format='win')
# print(st.__str__(extended=True))
#st[0].stats

#st_filtered = st.select(channel="bb00")
# st_filtered.plot()
# st_filtered = st.select(id="bb00")
#st.plot()

273 Trace(s) in Stream:
...01a6 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01a5 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01aa | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...00de | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01d5 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01df | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...022d | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01b1 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...022e | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...00dd | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...0100 | 2025-03-01T00:00:00.000000

memo: win形式データからobspyのStream型にする際に直井先生のwin_manipulate_toolsのコードを使用した。

In [41]:
import sys
sys.path.append("/workspaces/base-devcontainer/.reference/")
import win_manipulate_tools as wm
import time
import numpy as np
from obspy import Stream, Trace, UTCDateTime

ni = 1 # 繰り返し回数
start = time.perf_counter()
for i in range(ni):
    # 1990年代のデータを読む場合はcentury='１９'
    st = wm.read_winwave(f"./waveform/{win_file[1:]}", century='20', array_allocate_sec = 6200)
end = time.perf_counter()
print(f"実行時間: {end - start}")
print(st.__str__(extended=True))
#st.plot()


実行時間: 1.3282591400056845
273 Trace(s) in Stream:
...01A6 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01A5 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01AA | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...00DE | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01D5 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01DF | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...022D | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01B1 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...022E | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...00DD | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...0100 | 2

### Stream型データをMinIOに保存するPythonコード

In [35]:
import os
from io import BytesIO
import boto3
from botocore.client import Config
from obspy import Stream, read
import numpy as np
import json


class WaveformStorage:
    def __init__(self):
        """
        MinIOクライアントの初期化
        """
        self.s3_client = boto3.client(
            's3',
            endpoint_url='http://minio:9000',  # コンテナ名でアクセス
            aws_access_key_id='minioadmin',
            aws_secret_access_key='minioadmin',
            config=Config(signature_version='s3v4'),
            region_name='us-east-1'
        )
        
        # バケットが存在しない場合は作成
        self.bucket_name = 'waveform'
        try:
            self.s3_client.head_bucket(Bucket=self.bucket_name)
        except:
            self.s3_client.create_bucket(Bucket=self.bucket_name)
    
    def save_stream_npz(self, stream: Stream, key: str):
        """
        NPZ形式で保存（メタデータも含む）
        """
        try:
            with BytesIO() as buf:
                # トレースデータを個別に保存
                data_dict = {}
                stats = []
                
                for i, tr in enumerate(stream):
                    # 各トレースに固有のキーを割り当て
                    trace_key = f'trace_{i}'
                    data_dict[trace_key] = tr.data
                    
                    # メタデータを保存
                    stats.append({
                        'trace_key': trace_key,
                        'network': tr.stats.network,
                        'station': tr.stats.station,
                        'channel': tr.stats.channel,
                        'starttime': str(tr.stats.starttime),
                        'sampling_rate': tr.stats.sampling_rate,
                        'dtype': str(tr.data.dtype)
                    })
                
                # statsをJSON文字列に変換
                data_dict['stats'] = json.dumps(stats)
                
                # 保存
                np.savez_compressed(buf, **data_dict)
                buf.seek(0)
                self.s3_client.upload_fileobj(buf, self.bucket_name, f"{key}.npz")
                print(f"保存成功: {key}.npz")

        except Exception as e:
            print(f"保存エラー: {str(e)}")
            raise

    def load_stream_npz(self, key: str) -> Stream:
        """
        NPZ形式から読み込み
        """
        try:
            with BytesIO() as buf:
                self.s3_client.download_fileobj(self.bucket_name, f"{key}.npz", buf)
                buf.seek(0)
                data = np.load(buf, allow_pickle=True)
                
                # メタデータをJSONから復元
                stats = json.loads(str(data['stats']))
                
                # Streamの再構築
                stream = Stream()
                for stat in stats:
                    trace_data = data[stat['trace_key']]
                    trace = Trace(data=trace_data)
                    trace.stats.network = stat['network']
                    trace.stats.station = stat['station']
                    trace.stats.channel = stat['channel']
                    trace.stats.starttime = UTCDateTime(stat['starttime'])
                    trace.stats.sampling_rate = stat['sampling_rate']
                    stream += trace
                
                return stream
                
        except Exception as e:
            print(f"読み込みエラー: {str(e)}")
            raise




### 実際の使い方

In [32]:
# from waveform_storage import WaveformStorage

# ストレージクラスのインスタンス化
storage = WaveformStorage()

# Streamの保存
storage.save_stream_npz(st, f"{code}/2025/03/01/0000")


保存成功: 0203/2025/03/01/0000.npz


In [ ]:
# Streamの読み込み
loaded_stream = storage.load_stream_npz(f"{code}/2025/03/01/0000")
print(loaded_stream.__str__(extended=True))

273 Trace(s) in Stream:
...01A6 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01A5 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01AA | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...00DE | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01D5 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01DF | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...022D | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...01B1 | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...022E | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...00DD | 2025-03-01T00:00:00.000000Z - 2025-03-01T00:09:59.990000Z | 100.0 Hz, 60000 samples
...0100 | 2025-03-01T00:00:00.000000